In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/chat__bot'

Mounted at /content/drive


In [ ]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data_file=open(data_root + '/dataset.json').read()
data=json.loads(data_file)

In [ ]:
words=[]
classes=[]
data_X=[]
data_Y=[]

for intent in data["intents"]:
  for pattern in intent["examples"]:
    tokens=nltk.word_tokenize(pattern)
    words.extend(tokens)
    data_X.append(pattern)
    data_Y.append(intent["name"]),
  if intent["name"] not in classes:
    classes.append(intent["name"])

lemmatizer=WordNetLemmatizer()
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words=sorted(set(words))
classes=sorted(set(classes))

In [ ]:
training=[]
out_empty=[0]*len(classes)
for idx,doc in enumerate(data_X):
  bow=[]
  text=lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
  output_row=list(out_empty)
  output_row[classes.index(data_Y[idx])]=1
  training.append([bow,output_row])
random.shuffle(training)
training=np.array(training,dtype=object)
train_X=np.array(list(training[:,0]))
train_Y=np.array(list(training[:,1]))

In [ ]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_X[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]),activation="softmax"))
adam=tf.keras.optimizers.legacy.Adam(learning_rate=0.01,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X,y=train_Y,epochs=150,verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               7296      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                1040      
                                                                 
Total params: 16592 (64.81 KB)
Trainable params: 16592 (64.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
2/2

In [ ]:
def clean_text(text):
  tokens=nltk.word_tokenize(text)
  tokens=[lemmatizer.lemmatize(word) for word in tokens]
  return tokens

In [ ]:
def bag_of_words(text,vocab):
  tokens=clean_text(text)
  bow=[0] *len(vocab)
  for w in tokens:
    for idx,word in enumerate(vocab):
      if word== w:
        bow[idx]=1
  return np.array(bow)

In [ ]:
def pred_class(text,vocab,labels):
  bow=bag_of_words(text,vocab)
  result=model.predict(np.array([bow]))[0]
  thresh=0.5
  y_pred=[[indx,res] for indx,res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

In [ ]:
def get_response(intents_list,intents_json):
  if len(intents_list)==0:
    result = "Sorry! I don't understand."
  else:
    tag=intents_list[0]
    list_of_intents=intents_json["intents"]
    for i in list_of_intents:
      if i["name"] == tag:
        result=random.choice(i["responses"])
        break
  return result

In [ ]:
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
  message=input("")
  if message == "0":
    break
  intents=pred_class(message,words,classes)
  result=get_response(intents,data)
  print(result)

Press 0 if you don't want to chat with our ChatBot.
1/1 [==============================] - 0s 104ms/step
Hello! How can I assist you today?
1/1 [==============================] - 0s 42ms/step
Honey can be as effective as over-the-counter medicines for coughs, especially in children. However, avoid giving it to infants under 1 year due to the risk of food poisoning.
1/1 [==============================] - 0s 37ms/step
Sorry! I don't understand.
1/1 [==============================] - 0s 34ms/step
A hot bath can provide relief for conditions affecting muscles, bones, and tendons like arthritis, back pain, and joint pain. It promotes blood flow, but avoid excessive heat if you have a skin condition.
1/1 [==============================] - 0s 21ms/step
Peppermint oil might help with irritable bowel syndrome by reducing symptoms like cramps, bloating, and gas.
1/1 [==============================] - 0s 22ms/step
Ginger has been traditionally used to alleviate stomachaches, nausea, and menstrual